In [1]:
#importing all of the libraries that will be needed
import pandas as pd 
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
%matplotlib inline 
sns.set(color_codes=True)
import statistics as stats
from IPython.display import display, HTML

In [2]:
# Inserting some images that is related to the assignment topic

# Paths to my image files
image_paths = ['11.jpg', '12.jpg', '13.jpg']

# Generate HTML code to display images side by side
images_html = ''.join([f'<img src="{path}" style="width: 310px; margin: 0px; float: left;" />' for path in image_paths])

# Display HTML code
display(HTML(images_html))

In [16]:
#Read files from the URL into the pandas DataFrame
url = "airandtravel1.csv"
travel_df = pd.read_csv(url) 

## Air and Sea Travel 
Where did Ireland's Tourists Come From? This section will aim to identify the statistical data from countries that visited Ireland from 2016-2019. We will use data preperation methods such a `.head()` , `.shape`, `describe`, `info.()` etc to clean and sort the data first 

In [17]:
#Using .head() to get a sense for the data, its structure and content
travel_df.head()

,STATISTIC Label,Year,Month,Country,Direction,UNIT,VALUE
0,Air and Sea Travel,2010,January,Great Britain,Arrivals,Thousand,361.5
1,Air and Sea Travel,2010,January,Great Britain,Departures,Thousand,407.6
2,Air and Sea Travel,2010,January,Other UK (1),Arrivals,Thousand,0.9
3,Air and Sea Travel,2010,January,Other UK (1),Departures,Thousand,1.1
4,Air and Sea Travel,2010,January,Belgium,Arrivals,Thousand,12.5


In [18]:
travel_df.shape

(6084, 7)

In [19]:
# This provides me with information such as the datatypes. 
travel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6084 entries, 0 to 6083
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATISTIC Label  6084 non-null   object 
 1   Year             6084 non-null   int64  
 2   Month            6084 non-null   object 
 3   Country          6084 non-null   object 
 4   Direction        6084 non-null   object 
 5   UNIT             6084 non-null   object 
 6   VALUE            6080 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 332.8+ KB


In [20]:
travel_df.describe()
# because only value and year is an integer type (float) I am only returned with two coulmn of data for my statistical analsis 

,Year,VALUE
count,6084.000000,6080.000000
mean,2016.544379,136.604638
std,4.060471,316.126348
min,2010.000000,0.000000
25%,2013.000000,12.800000
50%,2017.000000,36.100000
75%,2020.000000,85.400000
max,2024.000000,2270.700000
